# Выбор локации для скважины

**Цель исследования**

- построить модель для определения региона, где добыча нефти принесёт наибольшую прибыль;
- проанализировать возможную прибыль и риски техникой Bootstrap.

**Описание данных:**

для проведения анализа представлены данные геологоразведки трёх регионов:

- */datasets/geo_data_0.csv*;
- */datasets/geo_data_1.csv*;
- */datasets/geo_data_2.csv*.

Ниже приведен перечень представленных даных: 

- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (по условиям проекта неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

**Условия задачи**:

- для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые);
- при разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки;
- бюджет на разработку скважин в регионе — 10 млрд рублей;
- при нынешних ценах один баррель сырья приносит 450 рублей дохода (доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей);
- после оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%; среди них выбирают регион с наибольшей средней прибылью;
- данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

**План работы:**

1. открыть файл с данными и изучить общую информацию;
2. выполнить предобработку данных;
3. обучить и проверить модель для каждого региона:
  
   3.1. разбить данные на обучающую и валидационную выборки в соотношении 75:25;
   
   3.2. обучить модель и сделать предсказания на валидационной выборке;
   
   3.3. сохранить предсказания и правильные ответы на валидационной выборке;
   
   3.4. напечатать на экране средний запас предсказанного сырья и RMSE модели;
   
   3.5. проанализировать результаты;
   
4. подготовить к расчёту прибыли:

   4.1. все ключевые значения для расчётов сохранить в отдельных переменных;
   
   4.2. рассчитать достаточный объём сырья для безубыточной разработки новой скважины и сравнить полученный объём сырья со средним запасом в каждом регионе; 
   
   4.3. написать выводы по этапу подготовки расчёта прибыли;
   
5. написать функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

   5.1. выбирать скважины с максимальными значениями предсказаний; 
   
   5.2. просуммировать целевое значение объёма сырья, соответствующее этим предсказаниям;
   
   5.3. рассчитать прибыль для полученного объёма сырья;
   
6. посчитать риски и прибыль для каждого региона:

   6.1. применить технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли;
   
   6.2. найти среднюю прибыль, 95%-й доверительный интервал и риск убытков (т.е. отрицательной прибыли);
   
   6.3. написать выводы: предложить регион для разработки скважин и обосновать выбор.


## Загрузка и подготовка данных

In [1]:
# импортируем нужные библиотеки
import pandas as pd
import seaborn as sns
sns.set()
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler 
from sklearn.pipeline import make_pipeline

In [2]:
# загружаем данные из файлов в датафреймы
reg1 = pd.read_csv('/datasets/geo_data_0.csv')
reg2 = pd.read_csv('/datasets/geo_data_1.csv')
reg3 = pd.read_csv('/datasets/geo_data_2.csv')
# выводим основную информацию о датафреймах с помощью метода info()
reg1.info()
print()
reg2.info()
print()
reg3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Nul

Таким образом, в каждом из наших датафреймов по 5 столбцов и 10 000 строк. Названия столбцов одинаковы для каждого датафрейма. Значения четырех столбцов относятся к вещественному типу (это f0, f1, f2 и product) и одного - к строковому (id). Теперь выведем первые 10 строк каждого из датафреймов на экран.

In [3]:
display(reg1.head(10))
display(reg2.head(10))
display(reg3.head(10))

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


В выведенных на экран фрагментах пропуски отсутствуют. Типы данных выбраны корректно. Очевидных противоречий либо маловероятных значений в представленных фрагментах не наблюдается. Теперь посмотрим распределение по каждому из столбцов датафреймов и проведем проверку на наличие пропусков и дубликатов. 

In [4]:
reg1.describe().T

,count,mean,std,min,25%,50%,75%,max
f0,100000.0,0.500419,0.871832,-1.408605,-0.072580,0.502360,1.073581,2.362331
f1,100000.0,0.250143,0.504433,-0.848218,-0.200881,0.250252,0.700646,1.343769
f2,100000.0,2.502647,3.248248,-12.088328,0.287748,2.515969,4.715088,16.003790
product,100000.0,92.500000,44.288691,0.000000,56.497507,91.849972,128.564089,185.364347


In [5]:
reg2.describe().T

,count,mean,std,min,25%,50%,75%,max
f0,100000.0,1.141296,8.965932,-31.609576,-6.298551,1.153055,8.621015,29.421755
f1,100000.0,-4.796579,5.119872,-26.358598,-8.267985,-4.813172,-1.332816,18.734063
f2,100000.0,2.494541,1.703572,-0.018144,1.000021,2.011479,3.999904,5.019721
product,100000.0,68.825000,45.944423,0.000000,26.953261,57.085625,107.813044,137.945408


In [6]:
reg3.describe().T

,count,mean,std,min,25%,50%,75%,max
f0,100000.0,0.002023,1.732045,-8.760004,-1.162288,0.009424,1.158535,7.238262
f1,100000.0,-0.002081,1.730417,-7.084020,-1.174820,-0.009482,1.163678,7.844801
f2,100000.0,2.495128,3.473445,-11.970335,0.130359,2.484236,4.858794,16.739402
product,100000.0,95.000000,44.749921,0.000000,59.450441,94.925613,130.595027,190.029838


Учитывая, что по условиям проекта неважно, что означают три признака точек (f0, f1, f2), явных противоречий либо маловероятных значений в представленных выше распределениях не наблюдается. Теперь проверим датафреймы на наличие пропусков.

In [7]:
# проверим количество пропусков в % 
pd.DataFrame(round(reg1.isna().mean()*100,).sort_values(ascending=False)).style.background_gradient('coolwarm')

,0
id,0.000000
f0,0.000000
f1,0.000000
f2,0.000000
product,0.000000


In [8]:
pd.DataFrame(round(reg2.isna().mean()*100,).sort_values(ascending=False)).style.background_gradient('coolwarm')

,0
id,0.000000
f0,0.000000
f1,0.000000
f2,0.000000
product,0.000000


In [9]:
pd.DataFrame(round(reg3.isna().mean()*100,).sort_values(ascending=False)).style.background_gradient('coolwarm')

,0
id,0.000000
f0,0.000000
f1,0.000000
f2,0.000000
product,0.000000


Как видим, во всех наших датафреймах отсутствуют пропуски. Теперь проверим на наличие дубликатов.

In [10]:
# проведем проверку на наличие дубликатов
print(reg1.duplicated().sum())
print(reg2.duplicated().sum())
print(reg3.duplicated().sum())

0
0
0


Таким образом, явные дубликаты в представленных датафреймах отсутствуют. При этом, учитывая, что все значения столбцов наших датафреймов (за исключением id) являются числовыми, осуществлять поиск неявных дубликатов не имеет смысла. На этом мы закончили этап подготовки данных и можем перейти к следующему шагу. 

Таким образом, на данном этапе мы выяснили следующее:

- в каждом из 3-х наших датафреймов по 5 столбцов и 10 000 строк. Названия столбцов одинаковы для каждого датафрейма. Значения четырех столбцов относятся к вещественному типу (это f0, f1, f2 и product) и одного - к строковому (id).
- Использованные типы данных корректны.
- Явные противоречия в данных, либо маловероятные значения отсутствуют.
- Пропуски данных отсутствуют.
- Явные и неявные дубликаты отсутствуют.

Теперь данные готовы для проведения исследования и мы можем приступить к обучению и проверке моделей.

## Обучение и проверка модели

Столбцы id датафреймов для дальнейшего исследования нам не понадобятся. Поэтому представляется целесообразным их удалить.

In [11]:
reg1.drop(columns='id', inplace=True)
reg2.drop(columns='id', inplace=True)
reg3.drop(columns='id', inplace=True)

In [12]:
display(reg1.head())
display(reg2.head())
display(reg3.head())

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261
2,6.263187,-5.948386,5.001160,134.766305
3,-13.081196,-11.506057,4.999415,137.945408
4,12.702195,-8.147433,5.004363,134.766305


,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697
2,0.194587,0.289035,-5.586433,62.871910
3,2.236060,-0.553760,0.930038,114.572842
4,-0.515993,1.716266,5.899011,149.600746


Теперь обучим и проверим модель для каждого региона. Для этого:

- разобьём данные на обучающую и валидационную выборки в соотношении 75:25;
- обучим модель и сделаем предсказания на валидационной выборке;
- сохраним предсказания и правильные ответы на валидационной выборке;
- напечатаем на экране средний запас предсказанного сырья и RMSE модели.

In [13]:
# создадим списки с наименованиями регионов и их датасетов
reg_names = ['1', '2', '3']
reg_df = [reg1, reg2, reg3]

# создадим словарь для хранения фактического и предсказанных значений набора valid
targets = {}

# пройдём циклом по парам имя-датасет
for name, df in zip(reg_names, reg_df):
    
    # создадим для каждой пары наборы признаков и целей
    target = df['product']
    features = df.drop('product', axis=1)
        
    # разобьём данные на обучающую и валидационную выборки в соотношении 75:25
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=1234)
    
    # создадим pipeline из скелера и модели линейной регрессии
    model = make_pipeline(StandardScaler(), LinearRegression())
    
    # обучим модель на обучающей выборке
    model.fit(features_train, target_train)
    
    # запишем фактические данные и предсказанные в словарь
    targets[name] = (target_valid, pd.Series(model.predict(features_valid), index=target_valid.index, name='product_predicted'))
    
    # напечатаем на экране средний запас предсказанного сырья и RMSE модели
    print()
    print(f'Для {name} региона:')
    print(f'Средний запас фактического сырья = {targets[name][0].mean():.2f} тыс. баррелей')
    print(f'Средний запас предсказанного сырья = {targets[name][1].mean():.2f} тыс. баррелей')
    print(f'Разница = {(1-targets[name][0].mean()/targets[name][1].mean()):.2%}')
    print(f'RMSE модели = {(mean_squared_error(target_valid, targets[name][1])**0.5):.2f} тыс. баррелей')
    print()
    print(f'Размер features_train: {features_train.shape}')
    print(f'Размер target_train: {target_train.shape}')
    print(f'Размер features_valid: {features_valid.shape}')
    print(f'Размер target_valid: {target_valid.shape}')


Для 1 региона:
Средний запас фактического сырья = 92.58 тыс. баррелей
Средний запас предсказанного сырья = 92.43 тыс. баррелей
Разница = -0.16%
RMSE модели = 37.56 тыс. баррелей

Размер features_train: (75000, 3)
Размер target_train: (75000,)
Размер features_valid: (25000, 3)
Размер target_valid: (25000,)

Для 2 региона:
Средний запас фактического сырья = 68.80 тыс. баррелей
Средний запас предсказанного сырья = 68.80 тыс. баррелей
Разница = -0.00%
RMSE модели = 0.89 тыс. баррелей

Размер features_train: (75000, 3)
Размер target_train: (75000,)
Размер features_valid: (25000, 3)
Размер target_valid: (25000,)

Для 3 региона:
Средний запас фактического сырья = 94.97 тыс. баррелей
Средний запас предсказанного сырья = 94.92 тыс. баррелей
Разница = -0.06%
RMSE модели = 40.10 тыс. баррелей

Размер features_train: (75000, 3)
Размер target_train: (75000,)
Размер features_valid: (25000, 3)
Размер target_valid: (25000,)


Таким образом, на 2-м шаге нашего исследования мы провели следующую работу:

- разбили данные на обучающую и валидационную выборки в соотношении 75:25;
- обучили модель (линейная регрессия) и сделали предсказания на валидационной выборке;
- сохранили предсказания и правильные ответы на валидационной выборке;
- напечатали на экране средний запас предсказанного сырья и RMSE модели.

Видим, что минимальная разница между прогнозом и фактом во 2-м регионе (0.00% при RMSE модели = 0.89 тыс. баррелей), а наибольшую разницу мы получили в регионе 1 (0.16% при RMSE модели = 37.56 тыс. баррелей). При этом необходимо учесть существенную разницу в запасах сырья между данными регионами. 

## Подготовка к расчёту прибыли

По условиям проекта при разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки. Бюджет на разработку скважин в регионе составляет 10 млрд. рублей. При нынешних ценах один баррель сырья приносит 450 рублей дохода (доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей). После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%; среди них выбирают регион с наибольшей средней прибылью.

In [14]:
#сохраняем ключевые значения для расчётов в отдельных переменных
points = 500
best_points = 200
budget = 10e9
barrel_price = 450e3
threshold = 0.025

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины и сравним полученный объём сырья со средним запасом в каждом регионе.

In [15]:
#рассчитаем точку безубыточности
break_even = budget / (barrel_price*best_points)
print(f'Достаточный объём сырья для безубыточной разработки = {break_even:.2f} тыс. баррелей')

Достаточный объём сырья для безубыточной разработки = 111.11 тыс. баррелей


In [16]:
#выведем на экран средний запас нефти в каждом из регионов
#а также % и количество скважин с объемом большим, чем точка безуюбыточности
for name, df in zip(reg_names, reg_df):
    print()
    print(f'Средний запас в {name} регионе = {df["product"].mean():.2f} тыс. баррелей.')
    print(f'Процент скважин с объёмом больше, чем точка безубыточности = {len(df.query("product > @break_even"))/len(df):.2%}')
    print(f'Количество скважин в объёмом больше, чем точка безубыточности = {len(df.query("product > @break_even"))}')


Средний запас в 1 регионе = 92.50 тыс. баррелей.
Процент скважин с объёмом больше, чем точка безубыточности = 36.58%
Количество скважин в объёмом больше, чем точка безубыточности = 36583

Средний запас в 2 регионе = 68.83 тыс. баррелей.
Процент скважин с объёмом больше, чем точка безубыточности = 16.54%
Количество скважин в объёмом больше, чем точка безубыточности = 16537

Средний запас в 3 регионе = 95.00 тыс. баррелей.
Процент скважин с объёмом больше, чем точка безубыточности = 38.18%
Количество скважин в объёмом больше, чем точка безубыточности = 38178


На данном этапе мы рассчитали точку безубыточности, а также средний запас нефти в каждом из регионов, процент и количество скважин с объемом большим, чем точка безубыточности.  
Мы выяснили, что достаточный объём сырья для безубыточной разработки составляет 111.11 тыс. баррелей на скважину. При этом ни в одном из регионов процент скважин с объёмом больше, чем точка безубыточности, не достигает и 40%.
Таким образом, чтобы вложения окупились, выбранные моделью скважины должны позволять добывать нефть в объемах, существенно превышающих 111.11 тыс. баррелей на скважину. 

## Расчёт прибыли и рисков 

На данном этапе исследования мы напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели. Для этого выберем скважины с максимальными значениями предсказаний, просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям, и рассчитаем прибыль для полученного объёма сырья.
После чего посчитаем риски и прибыль для каждого региона. Для этого применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли, найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков (т.е. отрицательной прибыли).

In [17]:
#создаем функцию для расчёта прибыли
def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)[:count]
    selected = target[probs_sorted.index]
    return selected.sum() * barrel_price

Теперь посчитаем риски и прибыль для каждого региона. Применим технику Bootstrap с 1000 выборок.

In [18]:
for name in reg_names:
    state = np.random.RandomState(12345)
    values = []
    for _ in range(1000):
        target_subsample = targets[name][0].sample(points, replace=True, random_state=state)
        probs_subsample = targets[name][1][target_subsample.index]
        target_subsample = target_subsample.reset_index(drop=True)
        probs_subsample = probs_subsample.reset_index(drop=True)
        values.append(revenue(target_subsample, probs_subsample, best_points) - budget)
        
    values = pd.Series(values)
        
    mean = values.mean() / 1e6
    lower = values.quantile(.025) / 1e6
    upper = values.quantile(.975) / 1e6
    
    risk = values.apply(lambda x: x < 0).sum() / len(values)

    print()
    print(f'Для {name} региона:')
    print(f'Прибыль с 200 лучших скважин, отобраных по предсказанию: {mean:.0f} млн. рублей.')
    print(f'Доверительный интервал лежит между {lower:.0f} - {upper:.0f} млн. рублей.')
    print(f'Риск убытков составляет: {risk:.2%}')


Для 1 региона:
Прибыль с 200 лучших скважин, отобраных по предсказанию: 387 млн. рублей.
Доверительный интервал лежит между -165 - 872 млн. рублей.
Риск убытков составляет: 6.90%

Для 2 региона:
Прибыль с 200 лучших скважин, отобраных по предсказанию: 434 млн. рублей.
Доверительный интервал лежит между 47 - 820 млн. рублей.
Риск убытков составляет: 1.30%

Для 3 региона:
Прибыль с 200 лучших скважин, отобраных по предсказанию: 332 млн. рублей.
Доверительный интервал лежит между -191 - 865 млн. рублей.
Риск убытков составляет: 10.10%


Как мы видим, наибольшая прибыль (434 млн. руб.) возможна в регионе 2. При этом в данном регионе также и наименьший риск убытков (он составляет 1.30%). Таким образом, для разработки скважин целесообразно рекомендовать именно 2-й регион. 

## Общий вывод:

Целью нашего исследования было построить модель для определения региона, где добыча нефти принесёт наибольшую прибыль, а также проанализировать возможную прибыль и риски техникой Bootstrap. Для проведения анализа были представлены данные геологоразведки трёх регионов:

- */datasets/geo_data_0.csv*;
- */datasets/geo_data_1.csv*;
- */datasets/geo_data_2.csv*.

Каждый из датафреймов состоит из 5 столбцов и 10 000 строк. Названия столбцов одинаковы для каждого датафрейма. Значения четырех столбцов относятся к вещественному типу (это f0, f1, f2 и product) и одного - к строковому (id). Использованные типы данных корректны. Явные противоречия в данных, либо маловероятные значения отсутствуют. Пропуски данных отсутствуют. Явные и неявные дубликаты отсутствуют.

На 2-м шаге нашего исследования мы провели следующую работу:

- разбили данные на обучающую и валидационную выборки в соотношении 75:25;
- обучили модель (линейная регрессия) и сделали предсказания на валидационной выборке;
- сохранили предсказания и правильные ответы на валидационной выборке;
- напечатали на экране средний запас предсказанного сырья и RMSE модели.

В результате установлено, что минимальная разница между прогнозом и фактом во 2-м регионе (0.00% при RMSE модели = 0.89 тыс. баррелей), а наибольшую разницу мы получили в регионе 1 (0.16% при RMSE модели = 37.56 тыс. баррелей).

Далее мы рассчитали точку безубыточности, а также средний запас нефти в каждом из регионов, процент и количество скважин с объемом большим, чем точка безубыточности. Мы выяснили, что достаточный объём сырья для безубыточной разработки составляет 111.11 тыс. баррелей на скважину. При этом ни в одном из регионов процент скважин с объёмом больше, чем точка безубыточности, не достигает и 40%. Таким образом, чтобы вложения окупились, выбранные моделью скважины должны позволять добывать нефть в объемах, существенно превышающих 111.11 тыс. баррелей на скважину.

При расчете прибыли и убытков установлено следующее:

1) **Для 1 региона:**

- прибыль с 200 лучших скважин, отобраных по предсказанию: 387 млн. рублей;
- доверительный интервал лежит между -165 - 872 млн. рублей;
- риск убытков составляет: 6.90%

2) **Для 2 региона:**

- прибыль с 200 лучших скважин, отобраных по предсказанию: 434 млн. рублей;
- доверительный интервал лежит между 47 - 820 млн. рублей;
- риск убытков составляет: 1.30%

3) **Для 3 региона:**

- прибыль с 200 лучших скважин, отобраных по предсказанию = 332 млн. рублей;
- доверительный интервал лежит между -191 - 865 млн. рублей;
- риск убытков составляет = 10.10%

Как мы видим, **наибольшая прибыль (434 млн. руб.) возможна в регионе 2. При этом в данном регионе также и наименьший риск убытков (он составляет 1.30%). Таким образом, для разработки скважин целесообразно рекомендовать именно 2-й регион.**